In [4]:
import numpy as np
from CustomApp.App import App
from SPFinder import SPFinder

In [ ]:
sp = SPFinder()
sp.read_10x(file='E://human/10X_Visium_hunter2021spatially_sample_C_data.h5ad')

In [ ]:
sp.app.run()

In [ ]:
from Algorithm.distance import get_exp_array
from PIL import Image
import numpy as np

from Algorithm.distribution import array_to_list
from sklearn import mixture


def load_image(file):
    image = Image.open(file)
    image_array = np.array(image.resize(get_exp_array(sp.adata, sp.adata.var.index[0]).shape))

    red_channel = image_array[:, :, 0]
    threshold = 255
    binary_matrix = (red_channel == threshold).astype(int)

    arr = np.array(binary_matrix, dtype=np.int32)
    result = array_to_list(arr)
    gmm = mixture.GaussianMixture(n_components=20, max_iter=200).fit(result)
    return gmm



In [ ]:
get_exp_array(sp.adata, sp.adata.var.index[0]).shape

In [ ]:
import seaborn as sns
sns.heatmap((red_channel == 2))

In [ ]:
def get_gmm(matrix, n_comp=10):
    arr = np.array(matrix, dtype=np.int32)
    result = array_to_list(arr)
    gmm = mixture.GaussianMixture(n_components=n_comp, max_iter=200).fit(result)
    return gmm


image = io.imread("E:/data/RegionC/RegionCplaqueHires.png")
cropped_image = cut_image(image)
resized = transform.resize(cropped_image, get_exp_array(adata, adata.var.index[0]).shape)
gaussian = filters.gaussian(resized, 3)
binary = np.where(gaussian > np.percentile(gaussian, 40), 1, 0)
gmm = get_gmm(binary, n_comp=20)

In [ ]:
x_min = sp.adata.obsm['spatial_real'][:,1].min()
x_max = sp.adata.obsm['spatial_real'][:,1].max()
y_min = sp.adata.obsm['spatial_real'][:,0].min()
y_max = sp.adata.obsm['spatial_real'][:,0].max()
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
# 打开图像文件
image = Image.open("E://human/GSM4838132_Visium_B_image.tif/GSM4838132_Visium_B_image.tif")  # 替换为你的图像文件路径

# 使用crop方法进行剪裁
cropped_image = image.crop((x_min, y_min, x_max, y_max))

# 保存剪裁后的图像
cropped_image.save("cropped_image.tif")  # 保存为新的文件，也可以覆盖原始文件

In [ ]:
sp.fit_pattern(n_top_genes=500, n_comp=50, min_cells=200)
sp.build_distance_array()

In [ ]:
sp.adata.var

In [ ]:
sp.cluster_gene(n_clusters=7, mds_components=20)

In [ ]:
from sklearn.metrics import davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import silhouette_score

db = {}
ch = {}
si = {}
for i in range(2, 12):
    sp.cluster_gene(n_clusters=i, mds_components=20)
    db[i] = 1 / davies_bouldin_score(sp.genes_distance_array, sp.kmeans_fit_result.labels_)
    ch[i] = calinski_harabasz_score(sp.genes_distance_array, sp.kmeans_fit_result.labels_)
    si[i] = silhouette_score(sp.genes_distance_array, sp.kmeans_fit_result.labels_)


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
index = pd.DataFrame([db,si,ch], index=['1/db','si','ch']).T
index_1 = (index-index.min()) / (index.max() - index.min())
sns.lineplot(index_1, markers=True)
plt.xticks(list(range(2, 12, 1)))
plt.xlabel("Number of Clusters")
plt.ylabel("Normalized Score")
plt.title("Evaluate Clustering Performance")
plt.show()

In [ ]:
sp.plot.plot_pattern(vmax=99, rotate=True, reverse_y=True, reverse_x=True)

In [ ]:
sp.plot.plot_genes(label=2, vmax=99, n_gene=20, cmap='viridis', plot_type='heatmap', rotate=True, reverse_y=True,
                   reverse_x=True)

In [ ]:
vlist = []
vdict={}
for i in sp.genes_patterns.keys():
    a = np.std(sp.genes_patterns[i].weights_)
    vlist.append(a)
    vdict[i]=a
sns.histplot(vlist)


In [ ]:
vdict['krt5']

In [ ]:
b = pd.DataFrame(vdict, index=['mse']).T

In [ ]:
a = sp.adata.var['variances_norm']

In [ ]:
merged = pd.merge(a, b, left_index=True, right_index=True, how='inner')

In [ ]:
merged

In [ ]:
sns.scatterplot(merged, y='variances_norm', x='mse')

In [ ]:
np.corrcoef(merged['variances_norm'], merged['mse'])[0, 1]

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6, 7))
sp.plot.plot_gene('pmela', reverse_x=True, rotate=True)

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk, ImageDraw


class App:
    def __init__(self):
        self.img = None
        self.img_on_canvas = None

        # Root windows
        self.root = tk.Tk()
        self.root.title("Region Annotation")

        # Create Canvas
        self.canvas = tk.Canvas(self.root, width=800, height=800)
        self.canvas.pack()
        self.canvas.bind("<Button-1>", self.annotate)
        self.canvas.bind("<Button-3>", self.annotate)

        # Create menu
        self.menu = tk.Menu(self.root)
        self.root.config(menu=self.menu)
        self.file_menu = tk.Menu(self.menu)
        self.menu.add_cascade(label="File", menu=self.file_menu)
        self.file_menu.add_command(label="Load image", command=self.open_image)
        self.file_menu.add_command(label="Save image", command=self.save_image)

        self.op_menu = tk.Menu(self.menu)
        self.menu.add_cascade(label="Edit", menu=self.op_menu)
        self.op_menu.add_command(label="Clean up annotations", command=self.clear_annotations)
        self.op_menu.add_command(label="Reset", command=self.reset)

        help_menu = tk.Menu(self.menu)
        self.menu.add_cascade(label="Help", menu=help_menu)
        help_menu.add_command(label="Help", command=self.show_help)
        help_menu.add_command(label="About", command=self.show_help)

        self.help_menu = tk.Menu(self.menu)

    def open_image(self):
        file_path = filedialog.askopenfilename()
        if file_path:
            self.root.title("Loading image... please wait.")
            self.img = Image.open(file_path)
            self.img = self.img.resize((800, 800), Image.ANTIALIAS)  # adjust img size
            self.img_on_canvas = ImageTk.PhotoImage(self.img)
            self.canvas.create_image(0, 0, anchor=tk.NW, image=self.img_on_canvas)
            self.root.title("Region Annotation")

    def annotate(self, event):
        x, y = event.x, event.y
        draw = ImageDraw.Draw(self.img)
        draw.ellipse((x - 5, y - 5, x + 5, y + 5), fill="red", outline="red")
        self.canvas.create_oval(x - 5, y - 5, x + 5, y + 5, fill="red", outline="red")

    def clear_annotations(self):
        self.canvas.delete("all")
        self.canvas.create_image(0, 0, anchor=tk.NW, image=self.img_on_canvas)

    def reset(self):
        self.canvas.delete("all")

    def save_image(self):
        file_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[('*', '.png')])
        if file_path:
            self.img.save(file_path)

    def show_help(self):
        help_text = "Usage: \n\n" \
                    "1. Use “File” -> “Load image” to load image.\n" \
                    "2. Click mouse over image to annotate image.\n" \
                    "3. Use “File” -> “Save image” to save the annotated image.\n\n" \
                    "Check the documentation for more details."
        messagebox.showinfo("Help", help_text)

    def run(self):
        self.root.mainloop()


a = App()
a.run()
